1. add 'is_festival' features
2. try to split features into 'is_8_20min', 'is_8_40min' etc. 
3. observe ratio between hour 7 and hour 8, ect. Check if the tendancy ratio vary rapidly among festival period
4. use ratio*volume to create some new features? \
5. deal with missing data: no data for some day

In [6]:
?np.corrcoef

In [7]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
pd.set_option('display.max_columns', 500)
import datetime
from __future__ import division
import seaborn as sns
from sklearn import linear_model

In [8]:
my_submission = pd.read_csv("predictions/RF_prediction_428.csv")

In [9]:
plt.plot(range(len(my_submission)), my_submission('volume'), '.')

TypeError: 'DataFrame' object is not callable

In [10]:
def MAPE(preds, outputs):
    preds = np.array(preds)
    outputs = np.array(outputs)
    return np.average(np.abs(outputs - preds) / outputs)

## Load data

In [11]:
combi_in = pd.read_csv("combi_in.csv")
combi_out = pd.read_csv("combi_out.csv")
combi_in = combi_in[combi_in['hour'].isin([6, 7, 8, 9, 15, 16, 17, 18])][['window_time', 'window_time_formatted', 'month',
                                                                    'day', 'hour', 'tollgate_id', 'direction', 'count',
                                                                          'minute']]
combi_out = combi_out[combi_out['hour'].isin([6, 7, 8, 9, 15, 16, 17, 18])][['window_time', 'window_time_formatted', 'month',
                                                                    'day', 'hour', 'tollgate_id', 'direction', 'count', 
                                                                            'minute']]


submission = pd.read_csv('submission_sample_volume.csv')

In [12]:
combi_in['rounded_min'] = combi_in['minute'] // 20 * 20
combi_out['rounded_min'] = combi_out['minute'] // 20 * 20

In [13]:
combi_in['window_time_formatted'] = combi_in['window_time_formatted'].apply(
                    lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')).tolist()
combi_out['window_time_formatted'] = combi_out['window_time_formatted'].apply(
                    lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')).tolist()

In [14]:
combi_in_1 = combi_in[combi_in['tollgate_id'] == 1]
combi_in_2 = combi_in[combi_in['tollgate_id'] == 2]
combi_in_3 = combi_in[combi_in['tollgate_id'] == 3]
combi_out_1 = combi_out[combi_out['tollgate_id'] == 1]
combi_out_3 = combi_out[combi_out['tollgate_id'] == 3]

## Move extreme values

In [15]:
def move_peak_values(df, hour):
    # Data in selected hour
    df_hour = df[df['hour'] == hour]
    # Normal data
    norm = np.percentile(df_hour['count'], 50)
    # Threshold value for filtering peak values
    threshold = 2 * norm
    # Peak values
    peak_values = df_hour[df_hour['count'] > threshold]['count']
    # Terminate in advance
    if len(peak_values) == 0:
        return df
    # Ratio used to adjust data
    ratio = np.percentile(peak_values, 50) / np.percentile(df_hour['count'], 50)
#     print ratio

    df.loc[(df['hour'] == hour) & (df['count'] > threshold), 'count'] = peak_values / ratio
    return df

def move_valley_values(df, hour):
    # Data in selected hour
    df_hour = df[df['hour'] == hour]
    # Normal data
    norm = np.percentile(df_hour['count'], 50)
    # Threshold value for filtering valley values
    threshold = norm / 2
    # Valley values
    valley_values = df_hour[df_hour['count'] < threshold]['count']
    # Terminate in advance
    if len(valley_values) == 0:
        return df
    # Ratio used to adjust data
    ratio = np.percentile(valley_values, 50) / np.percentile(df_hour['count'], 50)
#     print ratio

    df.loc[(df['hour'] == hour) & (df['count'] < threshold), 'count'] = valley_values / ratio
    return df

In [16]:
def move_extreme_values(df):
    hours = [8, 9, 17, 18]
    for hour in hours:
        df = move_peak_values(df, hour)
        df = move_valley_values(df, hour)
    return df

In [17]:
combi_in_1 = move_extreme_values(combi_in_1)
combi_in_2 = move_extreme_values(combi_in_2)
combi_in_3 = move_extreme_values(combi_in_3)
combi_out_1 = move_extreme_values(combi_out_1)
combi_out_3 = move_extreme_values(combi_out_3)

C:\Users\THINKPAD\Anaconda2\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Create features

In [18]:
datas = [combi_in_1, combi_in_2, combi_in_3, combi_out_1, combi_out_3]

#### Add 6 shifted time 

In [19]:
# Create shifted time windows
# eg: for window_time: '2016-9-18-8-20': 
#                      'ft_1': '2016-9-18-6-0'; 'ft_2': '2016-9-18-6-20'; 'ft_3': '2016-9-18-6-40'; 
#                      'ft_4': '2016-9-18-7-0'; 'ft_5': '2016-9-18-7-20'; 'ft_6': '2016-9-18-7-40'. 
# Note that '2016-9-18-8-0', '2016-9-18-8-20', '2016-9-18-8-40', '2016-9-18-9-0', '2016-9-18-9-20', '2016-9-18-9-40' share the
# same 'ft_1' to 'ft_6' 
def create_shifted_time(datas):
    for i in range(5):
        data = datas[i].copy()

        data['fake_hour'] = np.array([0] * len(data)).astype(str)
        data.loc[(data['hour'] == 8) | (data['hour'] ==9), 'fake_hour'] = str(6)
        data.loc[(data['hour'] == 17) | (data['hour'] == 18), 'fake_hour'] = str(15)
        data['fake_hour+1'] = (data['fake_hour'].astype(int) + 1).astype(str)

        data['year'] = data['window_time_formatted'].apply(lambda t : t.year).astype(str)
        data['month'] = data['window_time_formatted'].apply(lambda t : t.month).astype(str)
        data['day'] = data['window_time_formatted'].apply(lambda t : t.day).astype(str)

        features = ['ft_1', 'ft_2', 'ft_3', 'ft_4', 'ft_5', 'ft_6']
        slash = np.array(['-'] * len(data))
        data['ft_1'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour'] \
                        + slash + np.array([0] * len(data)).astype(str)
        data['ft_2'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour'] \
                        + slash + np.array([20] * len(data)).astype(str)
        data['ft_3'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour'] \
                        + slash + np.array([40] * len(data)).astype(str)
        data['ft_4'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour+1'] \
                        + slash + np.array([0] * len(data)).astype(str)
        data['ft_5'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour+1'] \
                        + slash + np.array([20] * len(data)).astype(str)
        data['ft_6'] = data['year'] + slash + data['month'] + slash + data['day'] + slash + data['fake_hour+1'] \
                        + slash + np.array([40] * len(data)).astype(str)

        for feature in features:
            data[feature] = data[feature].apply(lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M')).tolist()
        datas[i] = data
    return datas

In [20]:
datas = create_shifted_time(datas)

#### Add weather features

In [21]:
combi_in_1 = pd.read_csv("combi_in_1.csv")
combi_in_2 = pd.read_csv("combi_in_2.csv")
combi_in_3 = pd.read_csv("combi_in_3.csv")
combi_out_1 = pd.read_csv("combi_out_1.csv")
combi_out_3 = pd.read_csv("combi_out_3.csv")


weather_features = ['window_time_formatted', 
            'wind_direction_0.0', 'wind_direction_15.0', 'wind_direction_30.0', 'wind_direction_45.0', 'wind_direction_60.0', 
              'wind_direction_75.0', 'wind_direction_90.0', 'wind_direction_105.0', 'wind_direction_120.0', 
              'wind_direction_135.0', 'wind_direction_150.0', 'wind_direction_165.0', 'wind_direction_180.0', 
              'wind_direction_195.0', 'wind_direction_210.0', 'wind_direction_225.0', 'wind_direction_240.0', 
              'wind_direction_255.0', 'wind_direction_270.0', 'wind_direction_285.0', 'wind_direction_300.0', 
              'wind_direction_315.0', 'wind_direction_330.0', 'wind_direction_345.0', 'wind_direction_360.0', 
              'wind_speed_0.0', 'wind_speed_1.0', 'wind_speed_2.0', 'wind_speed_3.0', 'wind_speed_4.0', 'wind_speed_5.0', 
              'wind_speed_6.0', 'wind_speed_7.0', 'temperature_14.0', 'temperature_16.0', 'temperature_18.0', 
              'temperature_20.0', 'temperature_22.0', 'temperature_24.0', 'temperature_26.0', 'temperature_28.0', 
              'temperature_30.0', 'temperature_32.0', 'rel_humidity_40.0', 'rel_humidity_50.0', 'rel_humidity_60.0', 
              'rel_humidity_70.0', 'rel_humidity_80.0', 'rel_humidity_90.0', 'precipitation_0.0', 'precipitation_1.0', 
              'precipitation_2.0', 'precipitation_3.0', 'precipitation_4.0', 'precipitation_5.0']

combis = [combi_in_1, combi_in_2, combi_in_3, combi_out_1, combi_out_3]
for i in range(len(combis)):
    combi = combis[i]
    combi['window_time_formatted'] = combi['window_time_formatted'].apply(lambda t : 
                                                                         datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
    combis[i] = combi

for i in range(5):
    data = datas[i]
    combi = combis[i][weather_features]
    merged = pd.merge(left=data, right=combi, on='window_time_formatted', how='inner')
    datas[i] = merged

#### add 'ft_x_count' features

In [22]:
# Add 'count' feature for each of the 'ft_x' column
def create_prev_count(datas):
#     earlys = []
#     lates = []
    mers = []
    mer_lists = []
    ons = ['ft_1', 'ft_2', 'ft_3', 'ft_4', 'ft_5', 'ft_6']
    for k in range(5):
        data = datas[k]
        early = data[data['hour'].isin([6, 7, 15, 16])]
        late = data[data['hour'].isin([8, 9, 17, 18])]
        mer = late.copy()
#         earlys.append(early)
#         lates.append(late)
        for i in range(6):
            mer_list = pd.merge(left=late, right=early[['window_time_formatted', 'count']],
                       how='left', left_on=ons[i], right_on='window_time_formatted',suffixes=('', '_y'))['count_y'].tolist()
            mer_lists.append(mer_list)
#             print 'mer_list:', len(mer_list)
#             print 'mer length', len(mer)
            if len(mer_list) != len(mer):
                print k, i
                print ons[i] + '_count'
                print '----------------------'
#                 print mer
            feature = ons[i] + '_count'
            mer[feature] = mer_list


        mer['total_count'] = mer['ft_1_count'] + mer['ft_2_count'] + mer['ft_3_count'] + \
                                    mer['ft_4_count'] + mer['ft_5_count'] + mer['ft_6_count']
        mer['total_count'] = mer['total_count'] / 6
        mer['is_hour_8'] = (mer['hour'] == 8)
        mer['is_hour_9'] = (mer['hour'] == 9)
        mer['is_hour_17'] = (mer['hour'] == 17)
        mer['is_hour_18'] = (mer['hour'] == 18)
        mer['is_min_0'] = (mer['rounded_min'] == 0)
        mer['is_min_20'] = (mer['rounded_min'] == 20)
        mer['is_min_40'] = (mer['rounded_min'] == 40)
        mer['month_day'] = mer['month'].astype(str) + mer['day'].astype(str)

        # Fill NAN with mean here
        mer = mer.fillna(mer.mean())
        

        mers.append(mer)
    return mers

In [23]:
mers = create_prev_count(datas)

#### Add 'ft_x_ratio' features

In [24]:
def create_ft_ratio(datas, mers):
    
    mer_aves = []
    
    for i in range(5):
        data = datas[i]
        mer = mers[i]
        
        morning = data[data['hour'].astype(int).isin([6, 7])]
        afternoon = data[data['hour'].astype(int).isin([15, 16])]

#         hours_mor = [6, 7]
#         hours_aft = [15, 16]
#         minutes = [0, 20, 40]

        mor_count = []
        for hour in [6, 7]:
            for minute in [0, 20, 40]:
                part = morning[(morning['hour'] == hour) & (morning['rounded_min'] == minute)]
                count = np.average(part['count'])
                mor_count.append(count)

        aft_count = []
        for hour in [15, 16]:
            for minute in [0, 20, 40]:
                part = afternoon[(afternoon['hour'] == hour) & (afternoon['rounded_min'] == minute)]
                count = np.average(part['count'])
                aft_count.append(count)

        total_count = np.average(afternoon['count'])
        aft_count.append(total_count)

        ave_matrix = []
        for i in range(6):
            ave_n_count = [mor_count[i], mor_count[i], aft_count[i], aft_count[i]]
            ave_matrix.append(ave_n_count)

        ave_count = [np.average(mor_count), np.average(mor_count), np.average(aft_count), np.average(aft_count)]
        ave_matrix.append(ave_count)

        ave_df = pd.DataFrame({'hour':[8, 9, 17, 18], 'ave_ft_1':ave_matrix[0], 'ave_ft_2':ave_matrix[1], 
                              'ave_ft_3':ave_matrix[2], 'ave_ft_4':ave_matrix[3], 'ave_ft_5':ave_matrix[4],
                               'ave_ft_6':ave_matrix[5], 'ave_total_count':ave_matrix[6]})

        mer_ave = pd.merge(left=mer, right=ave_df, on='hour', how='left', suffixes=('', '_r_'))

        ratio_features = ['ft_1_ratio', 'ft_2_ratio', 'ft_3_ratio', 'ft_4_ratio', 'ft_5_ratio', 'ft_6_ratio', 'total_ratio']
        ave_featurers = ['ave_ft_1', 'ave_ft_2', 'ave_ft_3', 'ave_ft_4', 'ave_ft_5', 'ave_ft_6', 'ave_total_count']
        ft_features = ['ft_1_count', 'ft_2_count', 'ft_3_count', 'ft_4_count', 'ft_5_count', 'ft_6_count', 'total_count']

        for i in range(7):
            mer_ave[ratio_features[i]] = mer_ave[ft_features[i]] / mer_ave[ave_featurers[i]]
            
        mer_aves.append(mer_ave)
        
    return mer_aves

In [25]:
mers = create_ft_ratio(datas, mers)

In [26]:
def create_is_hour_min(mers):
    for i in range(len(mers)):
        df = mers[i]
        df['hour'] = df['hour']
        df['rounded_min'] =df['rounded_min']
        hours = [8, 9, 17, 18]
        minutes = [0, 20, 40]
        for hour in hours:
            for minute in minutes:
                new_feature = 'is_hour_' + str(hour) + '_min_' + str(minute)
                df[new_feature] = np.array([0] * len(df))
                df.loc[(df['hour'].astype(int) == hour) & (df['rounded_min'].astype(int) == minute), new_feature] = 1
        mers[i] = df
    return mers

1. find data that's in hour 7 and hour 8 
2. check which makes more sense, average or percentile 50
3. compute average of 'ft_1_count', for the moring time slot and afternoon time slot seperately
    get 14 results in total. (7 features here, including 'total_count_ave'
4. compute and add ratio as a new feature

## Determine features

In [28]:
features = ['ft_1_count', 'ft_2_count', 'ft_3_count', 'ft_4_count', 'ft_5_count', 'ft_6_count', 'total_count', 
           'is_hour_8', 'is_hour_9', 'is_hour_17', 'is_hour_18', 'is_min_0', 'is_min_20', 'is_min_40']

# features.remove('total_count')

In [29]:
# ratio_features = ['ft_1_ratio', 'ft_2_ratio', 'ft_3_ratio', 'ft_4_ratio', 'ft_5_ratio', 'ft_6_ratio', 'total_ratio']
# features = features + ratio_features

In [30]:
# features = features + weather_features
# features.remove('window_time_formatted')

In [31]:
# features = ['ft_1_count', 'ft_2_count', 'ft_3_count', 'ft_4_count', 'ft_5_count', 'ft_6_count', 'total_count', 
#            'is_hour_8_min_0', 'is_hour_8_min_20', 'is_hour_8_min_40',
#             'is_hour_9_min_0', 'is_hour_9_min_20', 'is_hour_9_min_40',
#             'is_hour_17_min_0', 'is_hour_17_min_20', 'is_hour_17_min_40', 
#              'is_hour_18_min_0', 'is_hour_18_min_20', 'is_hour_18_min_40']

## Split and normalize

In [32]:
data_sets = []
hours = [8, 9, 17, 18]
for mer in mers:
    for hour in hours:
        h_spec = mer[mer['hour'] == hour]
        data_sets.append(h_spec)

In [33]:
# from sklearn.preprocessing import normalize

# to_norm_featurs = ['ft_1_count', 'ft_2_count', 'ft_3_count', 'ft_4_count', 
#                   'ft_5_count', 'ft_6_count', 'total_count']

# for data in data_sets:
#     for feature in to_norm_featurs:
#         data[feature] = normalize(data[feature])[0]

## Split into training and test sets

In [34]:
import random

def split_train_test(data_sets, features):
    Xs_train_valid = []
    Ys_train_valid = []
    Xs_test = []
    Ys_test = []
    for i in range(20):
        df = data_sets[i]

        month_day = df['month_day'].unique()
        random.shuffle(month_day)
        test_size = int(0.25 * len(month_day))
        test_index = month_day[:test_size]
        train_index = month_day[test_size:]
    #     print len(test_index)
    #     print len(train_index)
        train = df[df['month_day'].isin(train_index)]
        test = df[df['month_day'].isin(test_index)]

        Xs_train_valid.append(train[features])
        Ys_train_valid.append(train['count'])
        Xs_test.append(test[features])
        Ys_test.append(test['count'])
    return (Xs_train_valid, Ys_train_valid, Xs_test, Ys_test)

In [35]:
(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test) = split_train_test(data_sets, features)

# Fit Models

## Linear Regression

In [36]:
from random import randint

def fit_model_LinearReg(X_train_valid, Y_train_valid):
    
    preds = []
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))

    alphas = np.arange(0.1, 10, 0.1)
    for a in alphas:
        
        
        kf = KFold(n_splits=5)
        mape_list = []
        for t_index, v_index in kf.split(X_train_valid):
            
            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]
            
            reg = linear_model.Ridge(alpha=a)
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape = MAPE(pred, valid_Y)
            mape_list.append(mape)
        mapes.append(np.average(mape_list))
        
    index = np.array(mapes).argmin()
    alpha = alphas[index]
    mape = mapes[index]
    reg = linear_model.Ridge(alpha=alpha)
    reg.fit(X_train_valid, Y_train_valid)
    
    return (alpha, mape, reg)

In [37]:
from sklearn import linear_model
from sklearn.model_selection import KFold

def train_LR_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=True):
    total_alphas = []
#     mapes_validation = []
    LR_models = []
    total_val_mapes = []
    total_test_mapes = []
    for i in range(20):
        (alpha, mape_valid, reg) = fit_model_LinearReg(Xs_train_valid[i], Ys_train_valid[i])

#         alphas.append(alpha)
#         mapes_validation.append(mape_valid)
        LR_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)
        total_alphas.append(alpha)
        
        if verbose:
            print ('alpha: {0:.4f}'.format(alpha), 'mape_val: {0:.4f}'.format(mape_valid),
                   'mape_test: {0:.4f}'.format(mape_test))
    ave_mape_valid = np.average(total_val_mapes)
    ave_mape_test = np.average(total_test_mapes)
    ave_alpha = np.average(total_alphas)
    print 'Validation mape:', '{0:.4f}'.format(ave_mape_valid)
    print 'Test mape:', '{0:.4f}'.format(ave_mape_test)
    print 'Alpha:', '{0:.4f}'.format(ave_alpha)
    print '-------------------End of training---------------------------'
    return (ave_mape_valid, ave_mape_test, LR_models, ave_alpha)

### Print linear regression model results

In [38]:
# without norm, without weather features
# average validation mape is: 0.20
# average test mape is: 0.22
# average alpha is: 6.75

# without norm, with weather features:
# average validation mape is: 0.1782
# average test mape is: 0.2627
# average alpha is: 4.6640

# without norm, without weather features, with 'total_count' / 6
# average validation mape is: 0.1975
# average test mape is: 0.2422
# average alpha is: 7.1140

# 

In [39]:
mape_valid_list = []
mape_test_list = []
alpha_list = []
LR_model_list = []
for i in range(5):
    (Xs_train_valid, Ys_train_valid, Xs_test, Ys_test) = split_train_test(data_sets, features)
    (mape_valid, mape_test, LR_models, alpha) \
            = train_LR_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=False)
    mape_valid_list.append(mape_valid)
    mape_test_list.append(mape_test)
    alpha_list.append(alpha)
    LR_model_list.append(LR_models)
print 'average validation mape is:', '{0:.4f}'.format(np.average(mape_valid_list))
print 'average test mape is:', '{0:.4f}'.format(np.average(mape_test_list))
print 'average alpha is:', '{0:.4f}'.format(np.average(alpha_list))

KeyboardInterrupt: 

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from random import randint


def fit_model_DT(X_train_valid, Y_train_valid):
    
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))
    
    kf = KFold(n_splits=10)
    
    
    for i in range(15):
        
        depth = 1+i
        mape_list = []
        
        for t_index, v_index in kf.split(X_train_valid):

            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]

            reg = DecisionTreeRegressor(max_depth=depth)
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape_list.append(MAPE(pred, valid_Y))
            
        mapes.append(np.average(mape_list))
    
    index = np.array(mapes).argmin()
    depth = index + 1
    mape = np.average(mape_list)
    reg = DecisionTreeRegressor(max_depth=depth)
    reg.fit(X_train_valid,Y_train_valid)
    
    return (depth, mape, reg)               

In [ ]:
from sklearn.model_selection import KFold

def train_DT_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=True):
    total_depths = []
    DT_models = []
    total_val_mapes = []
    total_test_mapes = []
    for i in range(20):
        (depth, mape_valid, reg) = fit_model_DT(Xs_train_valid[i], Ys_train_valid[i])

        DT_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)
        total_depths.append(depth)
        
        if verbose:
            print ('depth: {0:.4f}'.format(depth), 'mape_val: {0:.4f}'.format(mape_valid),
                   'mape_test: {0:.4f}'.format(mape_test))
    ave_mape_valid = np.average(total_val_mapes)
    ave_mape_test = np.average(total_test_mapes)
    ave_depth = np.average(total_depths)
    print 'Validation mape:', '{0:.4f}'.format(ave_mape_valid)
    print 'Test mape:', '{0:.4f}'.format(ave_mape_test)
    print 'Depth:', '{0:.4f}'.format(ave_depth)
    print '-------------------End of training---------------------------'
    return (ave_mape_valid, ave_mape_test, DT_models, ave_depth)

### Print Decision Tree Results

In [ ]:
# without norm, without weather features
# average validation mape is: 0.22
# average test mape is: 0.22
# average depth is: 3.19

# without norm, with weather features
# average validation mape is: 0.2270
# average test mape is: 0.2292
# average depth is: 3.2850

# without norm, without weather features, with 'total_count' / 6
# average validation mape is: 0.2322
# average test mape is: 0.2035
# average depth is: 2.8800

In [ ]:
mape_valid_list = []
mape_test_list = []
depth_list = []
DT_model_list = []
for i in range(5):
    (Xs_train_valid, Ys_train_valid, Xs_test, Ys_test) = split_train_test(data_sets, features)
    (mape_valid, mape_test, DT_models, depth) \
            = train_DT_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=False)
    mape_valid_list.append(mape_valid)
    mape_test_list.append(mape_test)
    depth_list.append(depth)
    DT_model_list.append(DT_models)
print 'average validation mape is:', '{0:.4f}'.format(np.average(mape_valid_list))
print 'average test mape is:', '{0:.4f}'.format(np.average(mape_test_list))
print 'average depth is:', '{0:.4f}'.format(np.average(depth_list))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from random import randint


def fit_model_RF(X_train_valid, Y_train_valid):
    
    initial_depth = 1
    depth = initial_depth
    initial_est = 5
    est = initial_est
    
    mapes = []
    regs = []
    
    seed = randint(0, 50)
    X_train_valid = X_train_valid.sample(frac=1, random_state=seed)
    Y_train_valid = Y_train_valid.sample(frac=1, random_state=seed)
    
    X_train_valid.index = range(len(X_train_valid))
    Y_train_valid.index = range(len(Y_train_valid))
    
    kf = KFold(n_splits=5)
    
    
    for i in range(10):
        
#         depth = depth + i
        mape_list = []
        est = est + 5 * i
        
        for t_index, v_index in kf.split(X_train_valid):

            train_X = X_train_valid.loc[t_index]
            valid_X = X_train_valid.loc[v_index]
            train_Y = Y_train_valid.loc[t_index]
            valid_Y = Y_train_valid.loc[v_index]

#             reg = RandomForestRegressor(max_depth=depth)
            reg = RandomForestRegressor(n_estimators=est)
            reg.fit(train_X, train_Y)
            pred = reg.predict(valid_X)
            mape_list.append(MAPE(pred, valid_Y))
            
        mapes.append(np.average(mape_list))
    
    index = np.array(mapes).argmin()
#     depth = index + 1
    est = initial_est + 5 * index
    mape = np.average(mape_list)
    reg = RandomForestRegressor(n_estimators=est)
    reg.fit(X_train_valid,Y_train_valid)
    
    return (est, mape, reg)               

In [ ]:
from sklearn.model_selection import KFold

def train_RF_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=True):
#     total_depths = []
    total_ests = []
    RF_models = []
    total_val_mapes = []
    total_test_mapes = []
    for i in range(20):
        (est, mape_valid, reg) = fit_model_RF(Xs_train_valid[i], Ys_train_valid[i])

        RF_models.append(reg)

        X_test = Xs_test[i]
        Y_test = Ys_test[i]
        pred = reg.predict(X_test)
        mape_test = MAPE(pred, Y_test)

        total_val_mapes.append(mape_valid)
        total_test_mapes.append(mape_test)
#         total_depths.append(depth)
        total_ests.append(est)
        
        if verbose:
#             print ('depth: {0:.4f}'.format(depth), 'mape_val: {0:.4f}'.format(mape_valid),
#                    'mape_test: {0:.4f}'.format(mape_test))
            print ('n_estimator: {0:.4f}'.format(est), 'mape_val: {0:.4f}'.format(mape_valid),
                   'mape_test: {0:.4f}'.format(mape_test))
    ave_mape_valid = np.average(total_val_mapes)
    ave_mape_test = np.average(total_test_mapes)
#     ave_depth = np.average(total_depths)
    ave_est = np.average(total_ests)
    print 'Validation mape:', '{0:.4f}'.format(ave_mape_valid)
    print 'Test mape:', '{0:.4f}'.format(ave_mape_test)
#     print 'Depth:', '{0:.4f}'.format(ave_depth)
    print 'n_estimators:', '{0:.4f}'.format(ave_est)
    print '-------------------End of training---------------------------'
    return (ave_mape_valid, ave_mape_test, RF_models, ave_est)

### Print Random Forest model results

In [ ]:
# without norm, without weather features
# average validation mape is: 0.19
# average test mape is: 0.19
# average depth is: 7.82

# without norm, with weather features
# average validation mape is: 0.1864
# average test mape is: 0.2212
# average depth is: 7.9000

# without norm, without weather features, with 'total_count' / 6
# average validation mape is: 0.1808
# average test mape is: 0.2045
# average depth is: 7.4400

In [ ]:
mape_valid_list = []
mape_test_list = []
# depth_list = []
n_estimator_list = []
RF_model_list = []
for i in range(1):
    (Xs_train_valid, Ys_train_valid, Xs_test, Ys_test) = split_train_test(data_sets, features)
    (mape_valid, mape_test, RF_models, est) \
            = train_RF_model_and_print(Xs_train_valid, Ys_train_valid, Xs_test, Ys_test, verbose=True)
    mape_valid_list.append(mape_valid)
    mape_test_list.append(mape_test)
#     depth_list.append(depth)
    n_estimator_list.append(est)
    RF_model_list.append(RF_models)
print 'average validation mape is:', '{0:.4f}'.format(np.average(mape_valid_list))
print 'average test mape is:', '{0:.4f}'.format(np.average(mape_test_list))
# print 'average depth is:', '{0:.4f}'.format(np.average(depth_list))
print 'average n_estimator is:', '{0:.4f}'.format(np.average(n_estimator_list))

In [ ]:
breakbopint here

In [ ]:
forest = RF_model_list[0][0]

In [ ]:
importances = forest.feature_importances_

In [ ]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
X = Xs_train_valid[0]

In [ ]:
create_RF_importance_df(forest, features)

In [ ]:
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

In [ ]:
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
breakbopint here

In [ ]:
def create_RF_importance_df(forest, features):
    importance = forest.feature_importances_
    df = pd.DataFrame(data={'importance':importance, 'feature':features})
    df.sort('importance', ascending=False, inplace=True)
    return df

In [ ]:
def create_coef_df(reg_model, features):
    coef_df = pd.DataFrame(data={'coef':reg_model.coef_, 'feature':features})
    coef_df.sort('coef', ascending=False, inplace=True)
    return coef_df

# Make submission

## Load data

In [40]:
sub_combi_in = pd.read_csv("sub_combi_in.csv")
sub_combi_out = pd.read_csv("sub_combi_out.csv")
test_combi_in = pd.read_csv('test_combi_in.csv')
test_combi_out = pd.read_csv('test_combi_out.csv')

In [41]:
sub_combi_in = pd.concat([sub_combi_in, test_combi_in])
sub_combi_out = pd.concat([sub_combi_out, test_combi_out])

useful_features = ['window_time', 'window_time_formatted', 'month', 'day', 'hour', 'tollgate_id', 'direction', 
                            'count', 'minute']
sub_combi_in = sub_combi_in[useful_features]
sub_combi_out = sub_combi_out[useful_features]

In [42]:
sub_combi_in['rounded_min'] = (sub_combi_in['minute'].astype(int) // 20 * 20).astype(str)
sub_combi_out['rounded_min'] = (sub_combi_out['minute'].astype(int) // 20 * 20).astype(str)

sub_combi_in['window_time_formatted'] = sub_combi_in['window_time_formatted'].apply(
                    lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')).tolist()
sub_combi_out['window_time_formatted'] = sub_combi_out['window_time_formatted'].apply(
                    lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')).tolist()

In [43]:
# # Merge data
# sub_combi_in = pd.merge(sub_combi_in, sub_gp_traj[['window_time_formatted', 'tollgate_id', 'direction']],
#              on=['window_time_formatted', 'tollgate_id', 'direction'], suffixes=('', '_y'), how='left')


# Split data by 'tollgate_id'
sub_combi_in_1 = sub_combi_in[sub_combi_in['tollgate_id'] == 1]
sub_combi_in_2 = sub_combi_in[sub_combi_in['tollgate_id'] == 2]
sub_combi_in_3 = sub_combi_in[sub_combi_in['tollgate_id'] == 3]
sub_combi_out_1 = sub_combi_out[sub_combi_out['tollgate_id'] == 1]
sub_combi_out_3 = sub_combi_out[sub_combi_out['tollgate_id'] == 3]

## Process data

In [44]:
sub_datas = [sub_combi_in_1, sub_combi_in_2, sub_combi_in_3, sub_combi_out_1, sub_combi_out_3]

In [45]:
sub_datas = create_shifted_time(sub_datas)
sub_mers = create_prev_count(sub_datas)

In [46]:
sub_data_sets = []
hours = [8, 9, 17, 18]
for mer in sub_mers:
    for hour in hours:
        h_spec = mer[mer['hour'] == hour]
        sub_data_sets.append(h_spec)

### Make prediction

In [ ]:
for i in range(20):
    model = LR_model_list[0][i]
    pred = model.predict(sub_data_sets[i][features])
    sub_data_sets[i]['preds'] = pred

# Predict using history data

In [107]:
sub_pred = pd.concat(sub_data_sets)

In [108]:
combi = pd.concat(mers)

In [109]:
sub_pred['preds'] = np.array([0] * len(sub_pred))

In [110]:
sub_pred.index = np.array(range(len(sub_pred)))

In [111]:
combi = combi[['day', 'hour', 'rounded_min', 'count', 'tollgate_id', 'direction']]

In [112]:
combi = combi.groupby(by=['hour', 'rounded_min', 'tollgate_id', 'direction']).agg('mean')
combi['hour'] = combi.index.get_level_values('hour')
combi['rounded_min'] = combi.index.get_level_values('rounded_min')
combi['tollgate_id'] = combi.index.get_level_values('tollgate_id')
combi['direction'] = combi.index.get_level_values('direction')

In [113]:
combi['minute'] = combi['rounded_min']

In [114]:
sub_pred = pd.merge(sub_pred, combi, how='left', on=['hour', 'minute', 'tollgate_id', 'direction'], suffixes=('', '_y'))

In [115]:
sub_pred['preds'] = sub_pred['count_y']

### Process sample submission dataframe

In [117]:
submission = pd.read_csv('submission_sample_volume.csv')

In [118]:
def submission_time_converter(df):
    df['time_formatted'] = df['time_window'].apply(
    lambda t : datetime.datetime.strptime(t.split(',')[0], '[%Y-%m-%d %H:%M:%S'))
    df['year'] = df['time_formatted'].apply(lambda t : t.year).astype(int)
    df['month'] = df['time_formatted'].apply(lambda t : t.month).astype(int)
    df['day'] = df['time_formatted'].apply(lambda t : t.day).astype(int)
    df['hour'] = df['time_formatted'].apply(lambda t : t.hour).astype(int)
    df['minute'] = df['time_formatted'].apply(lambda t : t.minute).astype(int)
    df['rounded_min'] = (df['minute'].astype(int) // 20 * 20).astype(int)
    slash = np.array(['-'] * len(df))
    df['window_time'] = df['year'].astype(str) + slash + df['month'].astype(str) + slash + df['day'].astype(str) \
                            + slash + df['hour'].astype(str) + slash + df['rounded_min'].astype(str)
    df['window_time_formatted'] = df['window_time'].apply(lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M'))
    return df

In [119]:
submission = submission_time_converter(submission)

In [120]:
# Merge sample data and prediction
my_submission = pd.merge(submission, sub_pred[['tollgate_id', 'direction', 'preds', 'window_time']],
                         on=['tollgate_id', 'direction', 'window_time'], how='left')

# Delete not useful data
my_submission = my_submission[['tollgate_id', 'direction', 'time_window', 'preds']]

# Rename column name
my_submission.rename(columns={'preds':'volume'}, inplace=True)

# Convert data type
my_submission['volume'] = my_submission['volume'].astype(int)

In [121]:
# Adjust volumn sequence
my_submission = my_submission[['tollgate_id', 'time_window', 'direction', 'volume']]

# Save data
my_submission.to_csv("predictions/history_prediction_503.csv", index=False)

# Load to check
my_submission = pd.read_csv("predictions/history_prediction_503.csv")

In [ ]:
ori = pd.read_csv('predictions/LR_prediction_423.csv')

In [ ]:
np.corrcoef(ori['volume'], my_submission['volume'])

In [ ]:
MAPE(ori['volume'], my_submission['volume'])

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(range(420), ori['volume'], '.', 
        range(420), my_submission['volume'], '*')

In [ ]:
RF_pred = pd.read_csv("predictions/RF_prediction_427_2.csv")
DT_pred = pd.read_csv("predictions/DT_prediction_427.csv")

In [ ]:
np.corrcoef(RF_pred['volume'], DT_pred['volume'])

In [ ]:
MAPE(RF_pred['volume'], DT_pred['volume'])

In [ ]:
np.average((RF_pred['volume'] - DT_pred['volume']) / DT_pred['volume'])

In [ ]:
np.average((DT_pred['volume'] - RF_pred['volume']) / RF_pred['volume'])

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(420), RF_pred['volume'], '.',
        range(420), DT_pred['volume'], '*')

# Observation

In [ ]:
df = datas[0]

In [ ]:
h6 = df[df['hour'].astype(int) == 6]
h7 = df[df['hour'].astype(int) == 7]
h8 = df[df['hour'].astype(int) == 8]
h9 = df[df['hour'].astype(int) == 9]
plt.figure(figsize=(12, 6))
print len(h7)==len(h8)
ratio = np.array(h8['count'].tolist()) / np.array(h7['count'].tolist())
plt.plot(range(len(h7)), ratio)

In [ ]:
def plot_ratio_trend(df, hour1, hour2):
    h1_df = df[df['hour'].astype(int) == hour1][['day', 'count']]
    h2_df = df[df['hour'].astype(int) == hour2][['day', 'count']]
    length = len(h1_df)
    ratio = np.array(h2_df['count'].tolist()) // np.array(h1_df['count'].tolist())
    
    plt.figure(figsize=(12, 6))
    plt.plot(length, ratio)

In [ ]:
df = datas[1]

In [ ]:
df.loc[df['hour'].astype(int).isin([7, 8])]

In [ ]:
plot_ratio_trend(datas[1], 7, 8)